<a href="https://colab.research.google.com/github/RiaGusmita/Indonesian-POS-Tagger-using-Flair/blob/main/EvaluateFlairForPOSTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/'My Drive'/Colab/Indonesian-POS-Tagger-using-Flair/

/content/gdrive/My Drive/Colab/Indonesian-POS-Tagger-using-Flair


In [4]:
!pip install flair

  Using cached https://files.pythonhosted.org/packages/f0/3a/1b46a0220d6176b22bcb9336619d1731301bc2c75fa926a9ef953e6e4d58/flair-0.8.0.post1-py3-none-any.whl
     |████████████████████████████████| 2.0MB 2.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 1.2MB 68.9MB/s 
     |████████████████████████████████| 983kB 69.9MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 798kB 64.1MB/s 
     |████████████████████████████████| 890kB 61.5MB/s 
     |████████████████████████████████| 3.2MB 61.3MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=1174395bbe5c25a10b2cce880290bfaf97c03958fba936252a260d7fadce46e7
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f

In [5]:
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings
from typing import List

In [ ]:
# 1. get the corpus
corpus = NLPTaskDataFetcher.load_corpus(NLPTask.UD_INDONESIAN)

# 2. what tag do we want to predict?
tag_type = 'upos'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
#print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),
    #WordEmbeddings('glove'),
    #BertEmbeddings('bert-base-multilingual-cased')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-universal-pos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=15)

In [ ]:
from flair.data import Sentence

sentence = Sentence('saya dan dia kemarin pergi ke pasar bersama untuk membeli jeruk')
tag_pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
tag_pos.predict(sentence)
print(sentence.to_tagged_string())

In [29]:
from tqdm import tqdm 
fileName = "/content/gdrive/MyDrive/Colab/Indonesian-POS-Tagger-using-Flair/Datasets/newFormatTrain.txt"
data = open(fileName,'r')
txt = data.read()

In [ ]:
### converting text in form of list of (words with their tags) ###
txt = txt.split('\n')

### removing DOCSTART (document header)
txt = [x for x in txt if x != '-DOCSTART- -X- -X- O']
### check ###
for i in range(10):
  print(txt[i])
  print('-'*10)

### Extracting Sentences ###
# Initialize empty list for storing words
words = []
# initialize empty list for storing sentences #
corpus = []

for i in tqdm(txt):
  ## if blank sentence encountered ##
  if i =='':
    ## previous words form a sentence ##
    corpus.append(' '.join(words))
    ## Refresh Word list ##
    words = []
  else:
   ## word at index 0 ##
    words.append(i.split()[0])

In [ ]:
for i in range(10):
  print(corpus[i])
  print('-'*10)

In [32]:
### Extracting POS ###
# Initialize empty list for storing word pos
w_pos = []
#initialize empty list for storing sentence pos #
POS = []
for i in tqdm(txt):
  ## blank sentence = new line ##
  if i =='':
    ## previous words form a sentence POS ##
    POS.append(' '.join(w_pos))
    ## Refresh words list ##
    w_pos = []
  else:
    ## pos tag from index 1 ##
    w_pos.append(i.split()[1])

100%|██████████| 163130/163130 [00:00<00:00, 1466454.37it/s]


In [ ]:
for i in range(10):
  print(corpus[i])
  print(POS[i])

In [ ]:
### Removing blanks form sentence and pos ###
corpus = [x for x in corpus if x!= '']
POS = [x for x in POS if x!= '']

### Check ###
for i in range(10):
  print(corpus[i])
  print(POS[i])

In [35]:
pos = SequenceTagger.load('resources/taggers/example-universal-pos/best-model.pt')
#for storing pos tagged string#
f_pos = []
## for every sentence ##
for i in tqdm(corpus):
  sentence = Sentence(i)
  pos.predict(sentence)
  ## append tagged sentence ##
  f_pos.append(sentence.to_tagged_string())

2021-03-30 22:58:53,771 loading file resources/taggers/example-universal-pos/best-model.pt


100%|██████████| 6720/6720 [00:20<00:00, 331.05it/s]


In [ ]:
for i in range(10):
  print(f_pos[i])
  print(corpus[i])

In [37]:
import re

### Extracting POS tags ###
## in every sentence by index ##
for i in tqdm(range(len(f_pos))):
  ## for every words ith sentence ##
  for j in corpus[i].split():
    ## replace that word from ith sentence in f_pos ##
    f_pos[i] = str(f_pos[i]).replace(j,"",1)

  ## Removing < > symbols ##
  for j in  ['<','>']:
    f_pos[i] = str(f_pos[i]).replace(j,"")

    ## removing redundant spaces ##
    f_pos[i] = re.sub(' +', ' ', str(f_pos[i]))
    f_pos[i] = str(f_pos[i]).lstrip()

100%|██████████| 6720/6720 [00:00<00:00, 30859.41it/s]


In [ ]:
for i in range(10):
  print(f_pos[i])
  print(corpus[i])

In [39]:
### Removing Symbols and redundant space ###

## in every sentence by index ##
for i in tqdm(range(len(corpus))):
  # Removing Symbols #
  corpus[i] = re.sub('[^a-zA-Z]', ' ', str(corpus[i]))
  POS[i] = re.sub('[^a-zA-Z]', ' ', str(POS[i]))
  f_pos[i] = re.sub('[^a-zA-Z]', ' ', str(f_pos[i]))  

  ## Removing HYPH SYM (they are for symbols) ##
  f_pos[i] = str(f_pos[i]).replace('HYPH',"")
  f_pos[i] = str(f_pos[i]).replace('SYM',"")
  POS[i] = str(POS[i]).replace('SYM',"")
  POS[i] = str(POS[i]).replace('HYPH',"")                       

  ## Removing redundant space ##
  POS[i] = re.sub(' +', ' ', str(POS[i]))
  f_pos[i] = re.sub(' +', ' ', str(f_pos[i]))
  corpus[i] = re.sub(' +', ' ', str(corpus[i]))  

100%|██████████| 6720/6720 [00:00<00:00, 27089.73it/s]


In [ ]:
for i in range(1000):
  print('corpus   '+corpus[i])
  print('actual   '+POS[i])  
  print('flair    '+f_pos[i])
  print('-'*50)

In [41]:
### EVALUATION FUNCTION ###
def eval(x,y):
  # correct match #
  count = 0
  #Total comparisons made# 
  comp = 0
  ## for every sentence index in dataset ##
  for i in range(len(x)):
    ## if the sentence length match ##
    if len(x[i].split()) == len(y[i].split()):
      ## compare each word ##
      for j in range(len(x[i].split())):
        if x[i][j] == y[i][j] :
          ## Match! ##
          count = count+1
          comp = comp + 1
        else:
          comp = comp + 1
  return (count/comp)*100

In [42]:
print("Flair Score ", eval(POS,f_pos))

Flair Score  55.42857142857143
